In [2]:
#built in librairies 
from tqdm import tqdm

# pip libraires
import numpy as np
import json
import pandas as pd


# visualization librairies
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Natural Language Processing Librairies
import nltk
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize
import re

# Visualization librairies
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Calculating libraries
import scipy as sp

#statistical librairies
import pingouin 

# Import libraries
import requests
from bs4 import BeautifulSoup
import pickle


#from genres import genre_wordsets
import csv
import datetime  
from dateutil.relativedelta import relativedelta
from utils.genres import additional_wordsets
from pandas_profiling import ProfileReport

In [ ]:
"""read table tvtropes and process the data to get rid of json format
"""
summaries=pd.read_csv('./data/plot_summaries.txt',sep="\t",header=None)
summaries.rename(columns={0:'wikipedia_movie_id',1:'Plot_summaries'},inplace=True)
movies=pd.read_table('./data/movie.metadata.tsv',header=None)
movies.rename(columns={0:'wikipedia_movie_id',1:'rebase_movie_ID',2:'Movie_name',3:'Movie_release',4:'Box_office_revenue',5:'Movie_runtime',6:'Movie_language',7:'Movie_country',8:'Movie_genre'},inplace=True)

df_summaries = pd.merge(summaries, movies[['wikipedia_movie_id', 'Movie_name']], on='wikipedia_movie_id')
pd.set_option('max_colwidth', 200)
df_summaries.head(20)

In [ ]:
import spacy
from spacy import displacy
import spacy_transformers

# Others
from functools import partial
#Vader
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

nlp_fast =spacy.load('en_core_web_sm')

nlp = spacy.load('en_core_web_trf')# en_core_web_sm, en_core_web_trf, xx_ent_wiki_sm

doc = nlp(" Marcus roams through the desert with Grubert where Vietnamese terrorist are roaming in Hgsvssd. in 1945 Vietmam was at war with the US as well as the Vietnames")# Display Entities
doc = nlp(df_summaries.iloc[7]['Plot_summaries'])
displacy.render(doc, style="ent")

In [ ]:
def clean_string(x:str)->list:
    """ Returns the lemmatized of every word in the input

    Args:
        x (str): Contains the input to be lemmatized

    Returns:
        list: a list of the words lemmatized
    """
    
    # Create a stemmer, the Lancaster Stemmer is used here
    doc = nlp_fast(x)
    # Return a list of Stemmed words
    return " ".join([token.lemma_.capitalize() for token in doc if not token.is_stop])

In [ ]:
import pandas as pd 
import geopandas as gpd 
import geopy 
import matplotlib.pyplot as plt
from geopy.extra.rate_limiter import RateLimiter
import requests
from time import sleep
from random import randint
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

user_agent = 'user_me_{}'.format(randint(10000,99999))
locator = Nominatim(user_agent=user_agent)

geocode = RateLimiter(locator.geocode, min_delay_seconds=1.2)

def find_country(country):
    address = geocode(country)
    if address is None:
        return "Other"
    lat = address.raw['lat']
    lon = address.raw['lon']
    url = f'https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json&accept-language=en&zoom=3'
    
    try:
        result = requests.get(url=url)
        result_json = result.json()
        return result_json['display_name']
    except Exception as e:
        return "Other"


In [ ]:
def dict_mean(dict_list):
    mean_dict = {}
    for key in dict_list[0].keys():
        mean_dict[key] = sum(d[key] for d in dict_list) / len(dict_list)
    return mean_dict

In [ ]:

max_val = len(df_summaries)
import collections, functools, operator

def find_movie_location_and_sentiment(summary):
    where = {}
    who = {}
    countries = {}
    
    doc = nlp(summary)
    vs_all = analyzer.polarity_scores(summary)

    for sent in doc.sents :
        
        phrase_sentiment = analyzer.polarity_scores(str(sent))

        for ent in sent.ents :
            if ent.label_ in ['GPE']:
                country = find_country(clean_string(ent.text))

                if clean_string(ent.text) in where:
                    where[clean_string(ent.text)].append(phrase_sentiment)
                else:
                    where[clean_string(ent.text)] = list([phrase_sentiment])

                if country in countries:
                    countries[country].append(phrase_sentiment)
                else:
                    countries[country] = list([phrase_sentiment])

            if ent.label_ in ['NORP']:
                if clean_string(ent.text) in who:
                    who[clean_string(ent.text)].append(phrase_sentiment)
                else:
                    who[clean_string(ent.text)] = list([phrase_sentiment])

    for key,value in where.items():
        dict_mean(value)
        where[key] = dict_mean(value)
    
    for key,value in who.items():
        who[key] = dict_mean(value)

    for key,value in countries.items():
        countries[key] = dict_mean(value)

    return pd.Series([who, where, countries, vs_all])
 
# function to apply 
df_summaries[['who', 'where', 'country', 'sentiment']] = df_summaries[6:8].Plot_summaries.apply(lambda x: find_movie_location_and_sentiment(x))

In [ ]:
import time
maxval = len(df_summaries)
step = 1000
i = 0
iteration = i*step
while i*step +step < maxval:
    print("Step ", i)
    user_agent = 'user_me_{}'.format(randint(10000,99999))
    locator = Nominatim(user_agent=user_agent)
    geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

    df_summaries_trunc = df_summaries[i*step:i*step + step]
    df_summaries_trunc[['who', 'where', 'country', 'sentiment']] = df_summaries_trunc.Plot_summaries.apply(lambda x: find_movie_location_and_sentiment(x))
    df_summaries_trunc.to_csv('tmp_stitches/country_sentiment_stitch_'+ str(i) + '.csv', index=True)
    i+=1

df_summaries_trunc = df_summaries[i*step:i*step + 204]
df_summaries_trunc[['who', 'where', 'country', 'sentiment']] = df_summaries_trunc.Plot_summaries.apply(lambda x: find_movie_location_and_sentiment(x))
df_summaries_trunc.to_csv('tmp_stitches/country_sentiment_stitch_'+ str(i) + '.csv', index=True)

In [12]:
import os
from ast import literal_eval

final_df = pd.read_csv('tmp_stitches/country_sentiment_stitch_'+ str(0) + '.csv',header=0, index_col=0)

for i in range(1,43):
    tmp_df = pd.read_csv('tmp_stitches/country_sentiment_stitch_'+ str(i) + '.csv',header=0, index_col=0)
    final_df = pd.concat([final_df, tmp_df], ignore_index=True)

print(len(final_df))
final_df[['who','where','country','sentiment']] = final_df[['who','where','country','sentiment']].applymap(literal_eval)

42204


In [13]:
final_df.to_csv('tmp_stitches/country_sentiment_stitch.csv', index=True)